In [2]:
import pandas as pd

In [5]:
path_to_Databases = "C:/Users/andre/OneDrive/Escritorio/Universidad/DM/codigoProyecto/"
title_complete_csv = path_to_Databases+"title_complete_csv.csv" 
title_complete = pd.read_csv(title_complete_csv)

In [6]:
title_complete

,tconst,primaryTitle,startYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0000009,Miss Jerry,1894,45,Romance,5.3,205
1,tt0000147,The Corbett-Fitzsimmons Fight,1897,100,"Documentary,News,Sport",5.3,480
2,tt0000502,Bohemios,1905,100,\N,4.1,15
3,tt0000574,The Story of the Kelly Gang,1906,70,"Action,Adventure,Biography",6.0,843
4,tt0000591,The Prodigal Son,1907,90,Drama,4.4,20
...,...,...,...,...,...,...,...
346786,tt9916362,Coven,2020,92,"Drama,History",6.4,5486
346787,tt9916428,The Secret of China,2019,\N,"Adventure,History,War",3.6,16
346788,tt9916460,Pink Taxi,2019,\N,Comedy,9.4,18
346789,tt9916538,Kuambil Lagi Hatiku,2019,123,Drama,8.6,7


In [26]:
resultado = title_complete[title_complete['genres'].str.contains('Horror')]


In [32]:
resultado2 = resultado[resultado['genres'].str.contains('Music')]

In [33]:
resultado2

,tconst,primaryTitle,startYear,runtimeMinutes,genres,averageRating,numVotes
6627,tt0025391,Life Returns,1934,63,"Drama,Horror,Music",3.9,238
13708,tt0036125,The Mad Ghoul,1943,65,"Horror,Music,Sci-Fi",5.8,828
13808,tt0036261,Phantom of the Opera,1943,92,"Drama,Horror,Music",6.4,7918
14115,tt0036715,The Climax,1944,86,"Horror,Music,Thriller",5.4,982
15239,tt0038338,The Beast with Five Fingers,1946,88,"Horror,Music,Mystery",6.5,2939
...,...,...,...,...,...,...,...
334241,tt8058874,We Summon the Darkness,2019,91,"Horror,Music,Thriller",5.2,11422
336486,tt8359848,Climax,2018,97,"Drama,Horror,Music",6.9,74883
344415,tt9509866,The Guy Who Didn't Like Musicals,2018,112,"Comedy,Horror,Musical",8.4,682
345706,tt9735758,Get Duked!,2019,87,"Comedy,Horror,Music",6.2,8209


In [15]:
count = 0
conjunto_Generos = set()
for i in range(len(title_complete['genres'])):
    generos = title_complete['genres'][i]
    generos = generos.split(",")
    for genero in generos:
        conjunto_Generos.add(genero)
print(len(conjunto_Generos))
print(conjunto_Generos)

28
{'History', 'Family', 'Game-Show', 'Comedy', 'Adventure', 'Thriller', 'Musical', 'Action', 'News', 'Western', 'Animation', 'Reality-TV', 'Music', 'Biography', 'Talk-Show', '\\N', 'Film-Noir', 'Crime', 'Drama', 'Adult', 'Mystery', 'Horror', 'Romance', 'Documentary', 'Sci-Fi', 'War', 'Fantasy', 'Sport'}


In [23]:
maximo = 0
for i in range(len(title_complete['genres'])):
    generos = title_complete['genres'][i]
    generos = generos.split(",")
    if(len(generos)> maximo):
        maximo=len(generos)
print(maximo)

3
